### Import Modules, Assign Data Path & Read into GDF

In [1]:
#Modules
import osmnx as ox 
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import matplotlib.pyplot as plt
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
ox.__version__
#Read Data into DataFrame
vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')
# Convert DataFrame to GeoDataFrame
vax_gdf = gpd.GeoDataFrame(vax_df, geometry=gpd.points_from_xy(vax_df['X'], vax_df['Y']))
#Reproject to UTM 17
vax_gdf.crs = 'EPSG:32617'

2022-03-09 11:32:17 Configured OSMnx 1.1.2
2022-03-09 11:32:17 HTTP response caching is on


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_63347/4099419668.py:16: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')


### Limit to Detroit Metro Area (Which we are defining as Wayne, Oakland, and Macomb County)

In [2]:
#limit by state to Michigan
vax_MI = vax_gdf[vax_gdf['State']== 'MI'] 
#Limit By County to Wayne, Oakland, and Macomb
vax_wayne = vax_MI[vax_MI['county']== 'Wayne']
vax_oakland = vax_MI[vax_MI['county']== 'Oakland'] 
vax_macomb = vax_MI[vax_MI['county']== 'Macomb']
#Agglomerate to one GDF of Detroit Metro
vax_detr_metro = vax_wayne 
vax_detr_metro = vax_detr_metro.append(vax_oakland)
vax_detr_metro = vax_detr_metro.append(vax_macomb)

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_63347/2395504573.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_oakland)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_63347/2395504573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_macomb)


In [3]:
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index('id') #could be here that somethign goes wrong

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
        edge_lines.append(edge_lookup)

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    new_iso = gpd.GeoSeries(all_gs).unary_union
        
    # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
    if infill:
        new_iso = Polygon(new_iso.exterior)
    isochrone_polys.append(new_iso)
    return isochrone_polys

In [4]:
def isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute, dist):
    network_type = network_type
    trip_time = trip_time
    travel_speed = travel_speed
    meters_per_minute = meters_per_minute
    dist = dist
    # download the street network
    G = ox.graph_from_point(place, dist, network_type=network_type)
    # find the centermost node and then project the graph to UTM
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    x, y = gdf_nodes['geometry'].unary_union.centroid.xy
    center_node = ox.get_nearest_node(G, (y[0], x[0]))
    G = ox.project_graph(G)
    for u, v, k, data in G.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute
    isochrone_polys = []
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)
    isochrone_polys = make_iso_polys(G, center_node, edge_buff=25, node_buff=0, infill=True)
    return isochrone_polys

In [5]:
#Set Variables
network_type = 'drive'
trip_time = 15 #in minutes 
travel_speed = 48.3 #walking speed in km/hour '''was 4.5 for walking & 48.3 for driving'''
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
detroit_polygons = gpd.GeoDataFrame(crs='EPSG:32617')
dist = meters_per_minute * trip_time

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_63347/152029120.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  detroit_polygons = gpd.GeoDataFrame(crs='EPSG:32617')


In [6]:
dist

12075.0

In [ ]:
for i in range(len(vax_detr_metro)):
    place = vax_detr_metro['Y'].iloc[i],vax_detr_metro['X'].iloc[i]
    isochrone_polys=isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute, dist)
    gdf_iso_poly = gpd.GeoDataFrame(index=[i], crs='EPSG:32617', geometry=isochrone_polys)
    detroit_polygons = detroit_polygons.append(gdf_iso_poly)
detroit_polygons.to_file('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Data/VaccineAccessShapefiles/15_min_drive_polys.shp')

2022-03-03 10:46:47 Created bbox 12075.0 m from (42.3963769002206, -83.4806078996323): 42.50496983073137,42.287783969709835,-83.33356222903255,-83.62765357023206
2022-03-03 10:46:47 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:46:47 Projected GeoDataFrame to epsg:4326
2022-03-03 10:46:47 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:46:47 Projected GeoDataFrame to epsg:4326
2022-03-03 10:46:47 Requesting data within polygon from API in 1 request(s)
2022-03-03 10:46:47 Resolved overpass-api.de to 178.63.48.217
2022-03-03 10:46:48 Pausing 0 seconds before making HTTP POST request
2022-03-03 10:46:48 Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A180%5D%3B%28way%5B%22highway%22%5D%5B%22area%22%21~%22yes%22%5D%5B%22access%22%21~%22private%22%5D%5B%22highway%22%21~%22abandoned%7Cbus_guideway%7Cconstruction%7Ccycleway%7Cmotor%7

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-03-03 10:49:38 Created nodes GeoDataFrame from graph
2022-03-03 10:49:42 Created nodes GeoDataFrame from graph
2022-03-03 10:49:53 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:50:15 Created edges GeoDataFrame from graph
2022-03-03 10:50:28 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:50:37 Created graph from node/edge GeoDataFrames
2022-03-03 10:50:37 Projected graph with 78860 nodes and 229108 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_30197/3347265067.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-03-03 10:53:57 Created bbox 12075.0 m from (42.3903256997602, -82.9147948003165): 42.498918630270964,42.28173276924943,-82.76776330655767,-83.06182629407533
2022-03-03 10:53:57 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:53:57 Projected GeoDataFrame to epsg:4326
2022-03-03 10:53:57 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 10:53:57 Projected GeoDataFrame to epsg:4326
2022-03-03 10:53:57 Requesting data within polygon from API in 1 request(s)
2022-03-03 10:53:57 Resolved overpass-api.de to 178.63.48.217
2022-03-03 10:53:57 Pausing 0 seconds before making HTTP POST request
2022-03-03 10:53:57 Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A180%5D%3B%28way%5B%22highway%22%5D%5B%22area%22%21~%22yes%22%5D%5B%22access%22%21~%22private%22%5D%5B%22highway%22%21~%22abandoned%7Cbus_guideway%7Cconstruction%7Ccycleway%7Cmotor%7

...

2022-03-03 21:10:34 Created nodes GeoDataFrame from graph
2022-03-03 21:10:42 Created nodes GeoDataFrame from graph
2022-03-03 21:10:59 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:11:43 Created edges GeoDataFrame from graph
2022-03-03 21:12:06 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:12:23 Created graph from node/edge GeoDataFrames
2022-03-03 21:12:23 Projected graph with 143533 nodes and 458924 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_30197/3347265067.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-03-03 21:17:17 Created bbox 12075.0 m from (42.2739005003063, -83.4888198002931): 42.382493430817064,42.16530756979553,-83.34206021958857,-83.63557938099763
2022-03-03 21:17:17 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:17:17 Projected GeoDataFrame to epsg:4326
2022-03-03 21:17:17 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:17:17 Projected GeoDataFrame to epsg:4326
2022-03-03 21:17:17 Requesting data within polygon from API in 1 request(s)
2022-03-03 21:17:17 Resolved overpass-api.de to 178.63.11.215
2022-03-03 21:17:18 Pausing 0 seconds before making HTTP POST request
2022-03-03 21:17:18 Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A180%5D%3B%28way%5B%22highway%22%5D%5B%22area%22%21~%22yes%22%5D%5B%22access%22%21~%22private%22%5D%5B%22highway%22%21~%22abandoned%7Cbus_guideway%7Cconstruction%7Ccycleway%7Cmotor%7

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-03-03 21:19:01 Created nodes GeoDataFrame from graph
2022-03-03 21:19:05 Created nodes GeoDataFrame from graph
2022-03-03 21:19:13 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:19:30 Created edges GeoDataFrame from graph
2022-03-03 21:19:41 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-03-03 21:19:47 Created graph from node/edge GeoDataFrames
2022-03-03 21:19:47 Projected graph with 65176 nodes and 189764 edges
